## Big Data w centrum logistycznym – analiza danych sprzedażowych i wysyłkowych w przedsiębiorstwie z branży AGD i RTV
### Informacje o zbiorze danych oraz opis

#### W pracy występują trzy zbiory danych, które są niezależne od siebie.
#### Pierwszy zbiór danch to informacje dotyczące sprzedaży (przedstawionej kwotowa) oraz ilości zamówień składanych w kanale e-commerce (B2C) w roku 2021.
###### Celem badania jest pokaznie najlepszych i najsłabszych okresów sprzedażowych pod względem kwotowym, jak i ilości zamówień w roku 2021.
###### Porównanie sprzedazy i wysyłki faktycznej versus plan stworzony w dziale sprzedaży.
###### Ukazanie najlepszych dni sprzedaży i wysłki towaru od strony operacji, jak i klienta finalnego.
###### Porónanie dwóch kanałów dystrubucji e-commerce (dwie strony internetowe działające niezależnie z tymi samymi artykułami).
###### Różnice w obsłudze logistycznej przez dwa osobne magazyny.

#### Drugi zbiór danych to informacje dotyczące wysyłki towaru w roku 2022 do sklepów. Brany pod uwagę jest podział na przewoźnika oraz miejsce dostarczenie towaru (sprzedaż detaliczna).
###### Celem w tym zbiorze danych jest pokazanie sklepów, które otrzymały najwięcej towaru w m3.
###### Rodzaj przewoźnika, jakim dostarczane są towary do sklepów.
###### Największego przewoźnika oraz najmniejszego.
###### Najlepszy oraz najsłabszy okres dotyczący ilości wysyłanego towaru do sklepów.
###### Najlepszy oraz najsłabszy dzień wysyłki towaru z podziałem na pracę magzynu oraz z informacją dla klienta indywidualnego, który dokonuje zakupu z odbiorem w sklepie stacjonarnym.

#### Trzeci zbiór danych to informacje dotyczące objętości wysłanego towaru w latach 2019-2022 z centrów logistycznych.
###### Celem jest porówanie objętości wysyłanej z magazynów logistycznych w latach 2019-2022.

In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyspark.sql.functions as f

In [0]:
%fs

ls /FileStore/tables

path,name,size,modificationTime
dbfs:/FileStore/tables/Dane_2021.csv,Dane_2021.csv,48874,1670767378000
dbfs:/FileStore/tables/Dane_2021a.csv,Dane_2021a.csv,33069,1670767904000
dbfs:/FileStore/tables/Dane_2021b.csv,Dane_2021b.csv,37070,1673009712000
dbfs:/FileStore/tables/Dane_2021c.csv,Dane_2021c.csv,37060,1673023818000
dbfs:/FileStore/tables/Dane_objetosc2-1.csv,Dane_objetosc2-1.csv,30490,1676396282000
dbfs:/FileStore/tables/Dane_objetosc2.csv,Dane_objetosc2.csv,27733,1676395956000
dbfs:/FileStore/tables/Dane_objetosc3.csv,Dane_objetosc3.csv,30490,1676398405000
dbfs:/FileStore/tables/Dane_objetosc4.csv,Dane_objetosc4.csv,30490,1676398731000
dbfs:/FileStore/tables/Dane_reg.csv,Dane_reg.csv,286,1673027063000
dbfs:/FileStore/tables/cykl_sprzedaży_produktów_NEONET_wer.csv,cykl_sprzedaży_produktów_NEONET_wer.csv,1891495,1670763213000


##### Wczytanie danych do analizy

In [0]:
### Dane dotyczące sprzedaży oraz ilości zamówień intenetnowych w kanale e-commerce w roku 2021
dane_sprzedaz = spark.read.format('csv').options(inferSchema=True, header=True, delimiter=';').load('/FileStore/tables/Dane_2021c.csv')

### Dane dotyczące wysłki towaru w roku 2022 z podziałem na przewoźnika oraz miejsce dostarczenie towaru (sprzedaż detaliczna)
detal_wysylki = spark.read.format('csv').options(inferSchema=True, header=True, delimiter=';').load('/FileStore/tables/detal_praca1.csv')

### Dane dotyczące objętości wysłanego towaru w latach 2019-2022
objetosc_wysylek = spark.read.format('csv').options(inferSchema=True, header=True, delimiter=';').load('/FileStore/tables/Dane_objetosc4.csv')

In [0]:
### Wyświtlenie danych sprzedażowych
display(dane_sprzedaz)

Data,Rok,Miesiac,Dzien,Dzien_tyg,NWW,DM,Total,Plan,Roznica_zamowien,Plan_sprzedazy,Wart_jednego_zam,Sprzedaz,Roznica,Wartosc_zam,Ilosc_Neo24,Ilosc_Neonet
2021-01-01,2021,1,1,5,42,20,62,287,-225,215481.68,3475.51,1407.31,-214074.37,22.7,0,60
2021-01-02,2021,1,2,6,2055,1199,3254,1779,1475,1334450.28,410.1,2260922.48,926472.2,694.81,1853,1262
2021-01-03,2021,1,3,7,2528,1029,3557,1944,1613,1457831.68,409.85,1886682.99,428851.31,530.41,2996,455
2021-01-04,2021,1,4,1,3775,1769,5544,3931,1613,2947975.66,531.74,2957662.85,9687.19,533.49,3828,1505
2021-01-05,2021,1,5,2,2412,849,3261,3954,-693,2965126.85,909.27,1796465.24,-1168661.61,550.89,1926,1189
2021-01-06,2021,1,6,3,1703,1158,2861,540,2321,405251.83,141.65,1054682.1,649430.27,368.64,2227,511
2021-01-07,2021,1,7,4,3424,1628,5052,3943,1109,2957284.57,585.37,3635948.28,678663.71,719.7,3758,1149
2021-01-08,2021,1,8,5,1506,655,2161,2155,6,1616112.63,747.85,1202384.57,-413728.06,556.4,1487,604
2021-01-09,2021,1,9,6,1456,43,1499,1779,-280,1334450.28,890.23,610938.51,-723511.77,407.56,715,723
2021-01-10,2021,1,10,7,2419,1092,3511,1944,1567,1457831.68,415.22,2005473.97,547642.29,571.2,2940,468


### Analiza i wizualizacja danych z wykorzystaniem narzędzi Python

In [0]:
dane_sprzedaz.groupBy('Miesiac','Total').count().explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[Miesiac#129, Total#134], functions=[finalmerge_count(merge count#301L) AS count(1)#294L])
   +- Exchange hashpartitioning(Miesiac#129, Total#134, 200), ENSURE_REQUIREMENTS, [plan_id=137]
      +- HashAggregate(keys=[Miesiac#129, Total#134], functions=[partial_count(1) AS count#301L])
         +- FileScan csv [Miesiac#129,Total#134] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[dbfs:/FileStore/tables/Dane_2021c.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Miesiac:int,Total:int>




In [0]:
### Pokazanie ilości zamówień w kanale e-commerce w każdy dzień roku 2021
display(dane_sprzedaz.select('Data','Total'))

Data,Total
2021-01-01,62
2021-01-02,3254
2021-01-03,3557
2021-01-04,5544
2021-01-05,3261
2021-01-06,2861
2021-01-07,5052
2021-01-08,2161
2021-01-09,1499
2021-01-10,3511


Output can only be rendered in Databricks

In [0]:
### Wyliczenie średniej ilości zamówień w roku 2021 w kanale e-commerce
dane_sprzedaz.select('Total').agg(f.round(f.mean('Total'),2).alias('Średnia_sprzedaz_w_roku_2021')).display()

Średnia_sprzedaz_w_roku_2021
2651.82


In [0]:
### Wyfiltrowanie totalu sprzedaży, który był większy niż średnia dla całego roku oraz ilość zamówień była większa od planu, wynik posortowany od największej ilości zamówień
dane_sprzedaz.filter('Total > 2651.82 and Roznica_zamowien > 0').sort('Total', ascending=False).display()

Data,Rok,Miesiac,Dzien,Dzien_tyg,NWW,DM,Total,Plan,Roznica_zamowien,Plan_sprzedazy,Wart_jednego_zam,Sprzedaz,Roznica,Wartosc_zam,Ilosc_Neo24,Ilosc_Neonet
2021-12-13,2021,12,13,1,6181,2575,8756,4900,3856,3674717.7,419.68,3738888.62,64170.92,427.01,6907,738
2021-12-20,2021,12,20,1,6255,1800,8055,4468,3587,3350727.79,415.98,4268282.75,917554.96,529.89,6534,774
2021-12-14,2021,12,14,2,5806,2129,7935,4389,3546,3292041.26,414.88,4244752.34,952711.08,534.94,6241,800
2021-11-29,2021,11,29,1,5710,2148,7858,7501,357,5625681.46,715.92,4864221.09,-761460.37,619.02,5513,2146
2021-12-06,2021,12,6,1,6149,1512,7661,5230,2431,3922368.64,511.99,4252641.14,330272.5,555.1,5903,763
2021-12-15,2021,12,15,3,5129,2268,7397,6257,1140,4692497.39,634.38,3476665.2,-1215832.19,470.01,5862,616
2021-12-16,2021,12,16,4,5417,1617,7034,5477,1557,4107724.95,583.98,3719534.04,-388190.91,528.79,5398,753
2021-12-01,2021,12,1,3,5097,1650,6747,6736,11,5051874.93,748.76,4857715.52,-194159.41,719.98,4171,1166
2021-11-30,2021,11,30,2,4923,1331,6254,5136,1118,3851858.68,615.9,4158365.88,306507.2,664.91,4237,1817
2021-12-09,2021,12,9,4,4604,1464,6068,4802,1266,3601133.56,593.46,3496283.15,-104850.41,576.18,4538,608


Output can only be rendered in Databricks

In [0]:
### Sprzedaż (kwotowo) w każdy dzień
display(dane_sprzedaz.select('Data','Sprzedaz'))

Data,Sprzedaz
2021-01-01,1407.31
2021-01-02,2260922.48
2021-01-03,1886682.99
2021-01-04,2957662.85
2021-01-05,1796465.24
2021-01-06,1054682.1
2021-01-07,3635948.28
2021-01-08,1202384.57
2021-01-09,610938.51
2021-01-10,2005473.97


In [0]:
### Wyliczenie średniej sprzedaży kowotowo roku 2021 w kanale e-commerce
dane_sprzedaz.select('Sprzedaz').agg(f.round(f.mean('Sprzedaz'),2).alias('Średnia_sprzedaz_kwotowo_w_roku_2021')).display()

Średnia_sprzedaz_kwotowo_w_roku_2021
1818454.77


In [0]:
### Wyfiltrowanie totalu sprzedaży, który był większy niż średnia dla całego roku oraz ilość zamówień była większa od planu, wynik posortowany od największej ilości zamówień
dane_sprzedaz.filter('Sprzedaz > 1818454.77 and Roznica > 0').sort('Sprzedaz', ascending=False).display()

Data,Rok,Miesiac,Dzien,Dzien_tyg,NWW,DM,Total,Plan,Roznica_zamowien,Plan_sprzedazy,Wart_jednego_zam,Sprzedaz,Roznica,Wartosc_zam,Ilosc_Neo24,Ilosc_Neonet
2021-12-20,2021,12,20,1,6255,1800,8055,4468,3587,3350727.79,415.98,4268282.75,917554.96,529.89,6534,774
2021-12-06,2021,12,6,1,6149,1512,7661,5230,2431,3922368.64,511.99,4252641.14,330272.5,555.1,5903,763
2021-12-14,2021,12,14,2,5806,2129,7935,4389,3546,3292041.26,414.88,4244752.34,952711.08,534.94,6241,800
2021-11-30,2021,11,30,2,4923,1331,6254,5136,1118,3851858.68,615.9,4158365.88,306507.2,664.91,4237,1817
2021-12-27,2021,12,27,1,3509,1178,4687,5063,-376,3797526.96,810.23,3868536.29,71009.33,825.38,3356,662
2021-12-13,2021,12,13,1,6181,2575,8756,4900,3856,3674717.7,419.68,3738888.62,64170.92,427.01,6907,738
2021-01-07,2021,1,7,4,3424,1628,5052,3943,1109,2957284.57,585.37,3635948.28,678663.71,719.7,3758,1149
2021-11-27,2021,11,27,6,3955,1154,5109,4323,786,3242051.67,634.58,3620037.84,377986.17,708.56,3186,1722
2021-01-20,2021,1,20,3,3287,1129,4416,3377,1039,2532823.93,573.56,3495144.5,962320.57,791.47,2477,1769
2021-11-15,2021,11,15,1,3552,1013,4565,4346,219,3259815.2,714.09,3472323.02,212507.82,760.64,3138,1269


Output can only be rendered in Databricks

In [0]:
### Średnia sprzedaż mieisięczna w roku 2021 w kanale e-commerce
dane_sprzedaz.groupBy('Miesiac')\
    .agg(f.round(f.mean('Sprzedaz'),2).alias('Średnia_sprzedaz_w_miesiacu')).sort('Miesiac', ascending=True).display()

Miesiac,Średnia_sprzedaz_w_miesiacu
1,1883386.64
2,1660325.18
3,1817175.55
4,1734035.65
5,1755378.67
6,1355806.92
7,1364990.47
8,1534576.73
9,1734035.65
10,1771341.55


Output can only be rendered in Databricks

In [0]:
### Ilość zamówień z podziałem na miesiące oraz najwieksza sprzedaż w każdym z miesięcy
dane_sprzedaz.groupBy('Miesiac')\
    .agg(f.round(f.sum('Total'),2).alias('Total_sprzedaz'),\
         f.max('Total').alias('Max_Total')).sort('Miesiac', ascending=True).display()

Miesiac,Total_sprzedaz,Max_Total
1,97183,5544
2,60176,3744
3,78371,4527
4,73243,4511
5,71060,4201
6,61669,3513
7,63294,3887
8,63734,3100
9,69874,3730
10,75610,4279


Output can only be rendered in Databricks

In [0]:
### Stworzenie przeliczenia numeracjnego z dniem tygodnia na nazwę dnia
Dni = spark.createDataFrame(
    [
      (1, 'Poniedziałek'),
        (2, 'Wtorek'),
        (3, 'Środa'),
        (4, 'Czwartek'),
        (5, 'Piątek'),
        (6, 'Sobota'),
        (7, 'Niedziela')        
    ],    
    ('id','Day')
)

In [0]:
### Wyświetlenie dni tygodnia
Dni.show()

+---+------------+
| id|         Day|
+---+------------+
|  1|Poniedziałek|
|  2|      Wtorek|
|  3|       Środa|
|  4|    Czwartek|
|  5|      Piątek|
|  6|      Sobota|
|  7|   Niedziela|
+---+------------+



In [0]:
### Dopasowanie nazwy dnia tygodnia do jego numeracyjnego odpowiednika w zbiorze danych
dane_sprzedaz = dane_sprzedaz.join(Dni, dane_sprzedaz.Dzien_tyg == Dni.id)

In [0]:
### Weryfikacja poprawności dopaowania nazwy dnia tygodnia do numeracji
dane_sprzedaz.select('Data','Dzien_tyg','Day').sort('Data', ascending=True).show(20)

+----------+---------+------------+
|      Data|Dzien_tyg|         Day|
+----------+---------+------------+
|2021-01-01|        5|      Piątek|
|2021-01-02|        6|      Sobota|
|2021-01-03|        7|   Niedziela|
|2021-01-04|        1|Poniedziałek|
|2021-01-05|        2|      Wtorek|
|2021-01-06|        3|       Środa|
|2021-01-07|        4|    Czwartek|
|2021-01-08|        5|      Piątek|
|2021-01-09|        6|      Sobota|
|2021-01-10|        7|   Niedziela|
|2021-01-11|        1|Poniedziałek|
|2021-01-12|        2|      Wtorek|
|2021-01-13|        3|       Środa|
|2021-01-14|        4|    Czwartek|
|2021-01-15|        5|      Piątek|
|2021-01-16|        6|      Sobota|
|2021-01-17|        7|   Niedziela|
|2021-01-18|        1|Poniedziałek|
|2021-01-19|        2|      Wtorek|
|2021-01-20|        3|       Środa|
+----------+---------+------------+
only showing top 20 rows



In [0]:
### Ilości zamówień w podziale na dzień tygodnia
dane_sprzedaz.groupBy('Day')\
    .agg(f.round(f.sum('Total'),2).alias('Total_sprzedaz'),\
         f.max('Total').alias('Max_Total')).sort('Total_sprzedaz', ascending=False).display()

Day,Total_sprzedaz,Max_Total
Poniedziałek,212087,8756
Wtorek,169745,7935
Środa,159116,7397
Czwartek,141817,7034
Niedziela,130672,5478
Piątek,92191,6631
Sobota,59635,5109


Output can only be rendered in Databricks

###### Podział zamówień na dzień ze słownym komentarzem o ilości zamówień

In [0]:
### stowrzenie funkcji do określenia słownego ilości zamówień
def Ilosc_zamowien(Zamowienia):
    if Zamowienia is None:
        return "brak danych"
    if Zamowienia < 1000:
        return "Bardzo mała ilość zamówień"
    elif Zamowienia >=1000 and Zamowienia < 2500:
        return "Mała ilość zamówień"
    elif Zamowienia >=2500 and Zamowienia < 3000:
        return "Średnia ilość zamówień"
    elif Zamowienia >=3000 and Zamowienia < 4500:
        return "Dobra ilość zamówień"
    elif Zamowienia >=4500 and Zamowienia < 6000:
        return "Bardzo dobra ilość zamówień"
    else:
        return 'Najlepsza ilość zamówień'
    

In [0]:
### Weryfikacja poprawności
Ilosc_zamowien(None)

Out[38]: 'brak danych'

In [0]:
### Pobranie odpowedniej biblioteki
from pyspark.sql.types import *

udf_Ilosc_zamowien = udf(Ilosc_zamowien, StringType())

In [0]:
### Dopasowanie w zbiorze danych
dane_sprzedaz = dane_sprzedaz.withColumn('Jaka_byla_wielkosc_zamowien', udf_Ilosc_zamowien(dane_sprzedaz.Total))

In [0]:
dane_sprzedaz.select('Data','Total','Jaka_byla_wielkosc_zamowien').sort('Data').display()

Data,Total,Jaka_byla_wielkosc_zamowien
2021-01-01,62,Bardzo mała ilość zamówień
2021-01-02,3254,Dobra ilość zamówień
2021-01-03,3557,Dobra ilość zamówień
2021-01-04,5544,Bardzo dobra ilość zamówień
2021-01-05,3261,Dobra ilość zamówień
2021-01-06,2861,Średnia ilość zamówień
2021-01-07,5052,Bardzo dobra ilość zamówień
2021-01-08,2161,Mała ilość zamówień
2021-01-09,1499,Mała ilość zamówień
2021-01-10,3511,Dobra ilość zamówień


Output can only be rendered in Databricks

In [0]:
### Weryfikacja czy ilość zamówień była zadowalająca
dane_sprzedaz.groupBy('Jaka_byla_wielkosc_zamowien').count().sort('count', ascending=False).display()

Jaka_byla_wielkosc_zamowien,count
Mała ilość zamówień,143
Dobra ilość zamówień,83
Średnia ilość zamówień,72
Bardzo mała ilość zamówień,34
Bardzo dobra ilość zamówień,20
Najlepsza ilość zamówień,12


Output can only be rendered in Databricks

In [0]:
### Weryfikacja średniej wartości zamówienia
dane_sprzedaz.groupBy('Miesiac')\
    .agg(f.round(f.mean('Wart_jednego_zam'),2).alias('Średnia_wartość_zamówienia'),\
         f.max('Total').alias('Max_Total')).sort('Miesiac', ascending=True).display()

Miesiac,Średnia_wartość_zamówienia,Max_Total
1,892.02,5544
2,907.82,3744
3,667.25,4527
4,940.28,4511
5,917.97,4201
6,1194.9,3513
7,822.67,3887
8,934.04,3100
9,838.87,3730
10,955.6,4279


Output can only be rendered in Databricks

#### 
Wyznacz sprzedaz (Sprzedaz) oraz śrendią wartosć zamówienia (Wartosc_zam) względem planu (Plan_sprzedazy) w każdy dzień (Data)

In [0]:
### Dane z podziałem miesiecznym wraz z sumą sprzedaży oraz sumą planu sprzedaży
dane_sprzedaz.groupBy('Miesiac')\
    .agg(f.round(f.mean('Wart_jednego_zam'),2).alias('Średnia_Wartosc_zam'),
         f.round(f.sum('Sprzedaz'),2).alias('Sprzedaz_miesieczna'),
         f.round(f.sum('Plan_sprzedazy'),2).alias('Plan_sprzedazy')).sort('Miesiac',ascending=True).display()

Miesiac,Średnia_Wartosc_zam,Sprzedaz_miesieczna,Plan_sprzedazy
1,892.02,5.838498592E7,5.857510109E7
2,907.82,4.482877996E7,5.080165781E7
3,667.25,5.633244195E7,5.144475009E7
4,940.28,5.202106945E7,6.589448493E7
5,917.97,5.441673881E7,5.560523038E7
6,1194.9,4.067420769E7,6.118970902E7
7,822.67,4.231470468E7,4.980062665E7
8,934.04,4.757187866E7,5.758195842E7
9,838.87,5.202106945E7,5.655738701E7
10,955.6,5.491158817E7,6.278209072E7


Output can only be rendered in Databricks

In [0]:
### Filtrowanie na dane sprzedazowe, które były większe niż zakładany plan
dane_sprzedaz.groupBy('Miesiac')\
    .agg(f.round(f.mean('Wart_jednego_zam'),2).alias('Średnia_Wartosc_zam'),
         f.round(f.sum('Sprzedaz'),2).alias('Sprzedaz_miesieczna'),
         f.round(f.sum('Plan_sprzedazy'),2).alias('Plan_sprzedazy')).filter('Sprzedaz_miesieczna > Plan_sprzedazy').sort('Miesiac',ascending=True).display()

Miesiac,Średnia_Wartosc_zam,Sprzedaz_miesieczna,Plan_sprzedazy
3,667.25,5.633244195E7,5.144475009E7


In [0]:
### Sprzedaż verus plan w dany dzień tygodnia
dane_sprzedaz.groupBy('Day')\
    .agg(f.round(f.mean('Wart_jednego_zam'),2).alias('Średnia_Wartosc_zam'),
         f.round(f.sum('Sprzedaz'),2).alias('Sprzedaz_miesieczna'),
         f.round(f.sum('Plan_sprzedazy'),2).alias('Plan_sprzedazy')).sort('Day', ascending=False).display()

Day,Średnia_Wartosc_zam,Sprzedaz_miesieczna,Plan_sprzedazy
Środa,899.12,1.178834598E8,1.4083183275E8
Wtorek,796.12,1.1810162903E8,1.2868909217E8
Sobota,1137.23,3.504898435E7,4.707691023E7
Poniedziałek,704.09,1.3468341452E8,1.4223062389E8
Piątek,948.95,6.318422055E7,7.425186865E7
Niedziela,705.05,8.44830856E7,8.115730505E7
Czwartek,929.78,1.0853274183E8,1.2777412167E8


Output can only be rendered in Databricks

In [0]:
### Ilości zamówień verus plan w każdym miesiacu
dane_sprzedaz.groupBy("Miesiac")\
    .agg(f.round(f.mean("Plan"),2).alias("Średni plan sprzedazy - ilości zamówień"),
    f.round(f.mean("Total"),2).alias("Średnia faktyczna sprzedaz - ilość zamówień"))\
.sort('Miesiac', ascending = True)\
.display()

Miesiac,Średni plan sprzedazy - ilości zamówień,Średnia faktyczna sprzedaz - ilość zamówień
1,2519.45,3134.94
2,2291.0,2228.74
3,2515.61,2528.1
4,1567.8,2441.43
5,2415.26,2292.26
6,2726.9,2055.63
7,2019.19,2041.74
8,2017.0,2055.94
9,2217.97,2329.13
10,2700.26,2439.03


Output can only be rendered in Databricks

In [0]:
dane_sprzedaz.select('Data','Sprzedaz').sort('Sprzedaz', ascending=False).display()

Data,Sprzedaz
2021-11-26,5369815.25
2021-11-29,4864221.09
2021-12-01,4857715.52
2021-12-20,4268282.75
2021-12-06,4252641.14
2021-12-14,4244752.34
2021-11-30,4158365.88
2021-12-27,3868536.29
2021-12-13,3738888.62
2021-12-16,3719534.04


In [0]:
### Porównaie zamówień pomiędzy pierwszym a drugim kanałem dystrybucji e-commerce
dane_sprzedaz.groupBy("Miesiac")\
    .agg(f.round(f.mean("Ilosc_Neo24"),2).alias("Średnia ilość zamówień przez pierwszy kanał sprzedaży e-com"),
         f.sum('Ilosc_Neo24').alias('ilość zamówień przez pierwszy kanał sprzedaży e-com'),
    f.round(f.mean("Ilosc_Neonet"),2).alias("Średnia ilość zamówień przez drugi kanał sprzedaży e-com"),
        f.sum("Ilosc_Neonet").alias('ilość zamówień przez drugi kanał sprzedaży e-com'))\
.sort('Miesiac', ascending = True).sort('Miesiac', ascending=True).display()

Miesiac,Średnia ilość zamówień przez pierwszy kanał sprzedaży e-com,ilość zamówień przez pierwszy kanał sprzedaży e-com,Średnia ilość zamówień przez drugi kanał sprzedaży e-com,ilość zamówień przez drugi kanał sprzedaży e-com
1,2112.77,65496,870.13,26974
2,1337.63,36116,820.37,22150
3,1571.87,48728,869.94,26968
4,1524.4,45732,826.37,24791
5,1426.84,44232,784.45,24318
6,1362.9,40887,574.83,17245
7,1434.23,44461,556.32,17246
8,1468.81,45533,582.55,18059
9,1570.67,47120,672.47,20174
10,1710.45,53024,617.87,19154


Output can only be rendered in Databricks

Output can only be rendered in Databricks

In [0]:
### Porównanie obsługi z dwóch centrów dystrybucji (NWW oraz DM)
dane_sprzedaz.groupBy('Miesiac')\
    .agg(f.sum('NWW').alias('Obsługa z magazynu NWW'),
        f.sum('DM').alias('Obsługa z magazynu DM'),
        f.sum('Total').alias('Całkowita obsługa'),
        f.round(f.mean('Total'),2).alias('Srednia w miesiacu'))\
.sort('Miesiac').display()

Miesiac,Obsługa z magazynu NWW,Obsługa z magazynu DM,Całkowita obsługa,Srednia w miesiacu
1,69429,27754,97183,3134.94
2,45918,14258,60176,2228.74
3,57767,20604,78371,2528.1
4,52982,20261,73243,2441.43
5,53114,17946,71060,2292.26
6,47745,13924,61669,2055.63
7,47804,15490,63294,2041.74
8,47228,16506,63734,2055.94
9,56501,13373,69874,2329.13
10,62715,12895,75610,2439.03


Output can only be rendered in Databricks

In [0]:
### Porównanie obsługi z dwóch centrów dystrybucji (NWW oraz DM) podział na dzień tygodnia
dane_sprzedaz.groupBy('Day')\
    .agg(f.sum('NWW').alias('Obsługa z magazynu NWW'),
        f.sum('DM').alias('Obsługa z magazynu DM'),
        f.sum('Total').alias('Całkowita obsługa'),
        f.round(f.mean('Total'),2).alias('Srednia w miesiacu'))\
.sort('Day').display()

Day,Obsługa z magazynu NWW,Obsługa z magazynu DM,Całkowita obsługa,Srednia w miesiacu
Czwartek,108987,32830,141817,2727.25
Niedziela,95725,34947,130672,2562.2
Piątek,72115,20076,92191,1739.45
Poniedziałek,157933,54154,212087,4078.6
Sobota,54100,5535,59635,1146.83
Wtorek,127549,42196,169745,3264.33
Środa,119846,39270,159116,3059.92


In [0]:
display(detal_wysylki)

Rok,Miesiac,Dzien,DataWysylki,Kod_pocztowy,Miasto,Adres,Objetosc,KanalDystrubucji,Akronim_kontrahenta,NumerWysylki,RodzajTransportu
2022,1,4,2022-01-04,05-082,Babice Nowe,Warszawska 195,1.85455,Detal,SKLEP_0N,WYS-01/00000043/2022,Suus
2022,1,4,2022-01-04,05-082,Babice Nowe,Warszawska 195,3.22505,Detal,SKLEP_0N,WYS-03/00000011/2022,Transport W�asny
2022,1,4,2022-01-04,05-100,Nowy Dw�r Mazowiecki,Warszawska 36,1.14336,Detal,SKLEP_NU,WYS-01/00000043/2022,Suus
2022,1,4,2022-01-04,05-230,Koby�ka,Napoleona 2F,0.39265,Detal,SKLEP_YK,WYS-01/00000043/2022,Suus
2022,1,4,2022-01-04,05-230,Koby�ka,Napoleona 2F,5.55001,Detal,SKLEP_YK,WYS-03/00000005/2022,Transport W�asny
2022,1,4,2022-01-04,05-822,Milanowek,Nowowiejska 2a,1.97913,Detal,SKLEP_MU,WYS-01/00000043/2022,Suus
2022,1,4,2022-01-04,05-822,Milanowek,Nowowiejska 2a,1.35098,Detal,SKLEP_MU,WYS-03/00000028/2022,Suus
2022,1,4,2022-01-04,05-822,Milanowek,Nowowiejska 2a,1.38806,Detal,SKLEP_MU,WYS-03/00000011/2022,Transport W�asny
2022,1,4,2022-01-04,06-100,Pu�tusk,Jana Paw�a II 7,0.85335,Detal,SKLEP_P0,WYS-01/00000043/2022,Suus
2022,1,4,2022-01-04,06-100,Pu�tusk,Jana Paw�a II 7,2.78512,Detal,SKLEP_P0,WYS-03/00000005/2022,Transport W�asny


In [0]:
detal_wysylki.groupBy('Miasto')\
    .agg(f.round(f.mean('Objetosc'),2).alias('Średnia dzienna ilosc wyyslek'),\
         f.max('Objetosc').alias('Max_Objetosc'))\
.sort('Średnia dzienna ilosc wyyslek', ascending = False).display()

Miasto,Średnia dzienna ilosc wyyslek,Max_Objetosc
Sierpc,11.29,15.95647
Kartuzy,3.47,14.67038
Chorz�w,3.17,14.76277
D�bno,3.01,10.87764
Dzier�oni�w,2.74,13.28927
�ary,2.59,16.85457
�nin,2.55,31.85299
Hrubieszow,2.53,35.34791
Przeworsk,2.47,16.162635
Ostrowiec �wi�tokrzyski,2.44,9.80303


In [0]:
### Sklepy z najwięszką sumą objętości wysyłek otrzymanych z magazynu
detal_wysylki.groupBy(f.substring('Akronim_kontrahenta',7,2))\
    .agg(f.round(f.sum('Objetosc'),2).alias('Suma objętości wysłanych do danego sklepu'))\
.sort('Suma objętości wysłanych do danego sklepu', ascending=False).withColumnRenamed('substring(Akronim_kontrahenta, 7, 2)','Skrót nazwy sklepu').display()

Skrót nazwy sklepu,Suma objętości wysłanych do danego sklepu
QI,1526.86
DQ,1438.86
DI,1409.1
NC,1334.14
3Z,1265.58
CY,1259.52
A2,1212.49
CU,1205.34
ZV,1186.89
4Y,1170.78


In [0]:
detal_wysylki.groupBy('RodzajTransportu')\
    .agg(f.round(f.sum('Objetosc'),2).alias('Suma wysłanych przesyłek z podziałem na przewoźnika'),\
         f.max('Objetosc').alias('Max_Objetosc'))\
.sort('RodzajTransportu', ascending = True).display()

RodzajTransportu,Suma wysłanych przesyłek z podziałem na przewoźnika,Max_Objetosc
DPD,781.72,0.95202
Spedimex,24319.22,19.7484
Suus,61426.08,37.4175307
Transport W�asny,87193.34,28.405531


In [0]:
### Podział wysyłki na przewoźnika w każdym miesiącu
detal_wysylki.groupBy(['Miesiac','RodzajTransportu'])\
    .agg(f.round(f.sum('Objetosc'),2).alias('Suma wysłanych przesyłek z podziałem na przewoźnika'))\
        .sort('Miesiac', ascending = True).display()

Miesiac,RodzajTransportu,Suma wysłanych przesyłek z podziałem na przewoźnika
1,Spedimex,1929.31
1,Transport W�asny,6480.88
1,DPD,51.12
1,Suus,4616.89
2,Spedimex,1475.54
2,DPD,76.6
2,Transport W�asny,5455.46
2,Suus,4004.2
3,Spedimex,1795.77
3,Suus,5146.5


Output can only be rendered in Databricks

In [0]:
detal_wysylki.groupBy(['RodzajTransportu','Miasto'])\
    .agg(f.round(f.sum('Objetosc'),2).alias('Suma wysłanych przesyłek z podziałem na przewoźnika'),\
        f.round(f.max('Objetosc'),2).alias('Max objętość'),\
         f.round(f.min('Objetosc'),2).alias('Min objętość'),\
        f.round(f.mean('Objetosc'),2).alias('Średnia Objetoc wysyłek'))\
        .sort('Średnia Objetoc wysyłek', ascending = False).display()

RodzajTransportu,Miasto,Suma wysłanych przesyłek z podziałem na przewoźnika,Max objętość,Min objętość,Średnia Objetoc wysyłek
Transport W�asny,D�bno,30.69,14.49,2.12,7.67
Transport W�asny,Sierpc,185.36,15.96,0.05,7.41
Transport W�asny,Chorz�w,288.63,28.41,0.75,6.56
Transport W�asny,Dzier�oni�w,1241.83,13.62,0.64,4.44
Transport W�asny,Kartuzy,1289.31,14.87,0.0,4.4
Transport W�asny,Kielce,134.47,9.24,1.4,4.34
Transport W�asny,�ary,1136.73,16.85,0.44,3.95
Transport W�asny,Cieszyn,1094.72,13.08,0.4,3.95
Transport W�asny,Lubrza,230.28,15.37,0.0,3.78
Transport W�asny,Wadowice,157.45,7.55,0.34,3.75


In [0]:
detal_wysylki = detal_wysylki.join(Dni, detal_wysylki.Dzien == Dni.id)

In [0]:
detal_wysylki.groupBy('Day')\
    .agg(f.round(f.sum('Objetosc'),2).alias('Dzienna_Objetosc_Wysylek'),\
         f.max('Objetosc').alias('Max_Objetosc'),
        f.round(f.mean('Objetosc'),2).alias('Średnia Objetoc wysyłek'))\
.sort('Dzienna_Objetosc_Wysylek', ascending = True).display()

Day,Dzienna_Objetosc_Wysylek,Max_Objetosc,Średnia Objetoc wysyłek
Środa,4011.84,23.8663,1.32
Niedziela,5105.97,16.162635,1.4
Poniedziałek,5122.06,35.34791,1.43
Piątek,5341.16,14.76277,1.38
Sobota,5547.48,15.95647,1.65
Czwartek,5641.86,29.17496,1.46
Wtorek,5922.21,31.85299,1.49


Output can only be rendered in Databricks

In [0]:
detal_wysylki.groupBy(['Day','RodzajTransportu'])\
    .agg(f.round(f.sum('Objetosc'),2).alias('Dzienna_Objetosc_Wysylek'),\
         f.max('Objetosc').alias('Max_Objetosc'),
        f.round(f.mean('Objetosc'),2).alias('Średnia Objetoc wysyłek'))\
.sort('Dzienna_Objetosc_Wysylek', ascending = True).display()

Day,RodzajTransportu,Dzienna_Objetosc_Wysylek,Max_Objetosc,Średnia Objetoc wysyłek
Poniedziałek,DPD,19.42,0.34354,0.04
Sobota,DPD,19.95,0.84172,0.04
Niedziela,DPD,20.21,0.50497,0.03
Środa,DPD,20.73,0.76064,0.05
Wtorek,DPD,25.31,0.58708,0.05
Piątek,DPD,26.23,0.36624,0.04
Czwartek,DPD,27.02,0.53028,0.04
Środa,Spedimex,549.63,7.20927,1.38
Poniedziałek,Spedimex,684.27,11.36176,1.52
Niedziela,Spedimex,704.42,16.162635,1.61


Output can only be rendered in Databricks

Output can only be rendered in Databricks

In [0]:
display(objetosc_wysylek)

Data,Objetosc_wysylek,Dzien_tyg
2019-01-01,124.56,2
2019-01-02,639.45,3
2019-01-03,613.56,4
2019-01-04,602.23,5
2019-01-05,589.66,6
2019-01-06,45.6,7
2019-01-07,847.45,1
2019-01-08,856.65,2
2019-01-09,678.56,3
2019-01-10,641.25,4


In [0]:
objetosc_wysylek = objetosc_wysylek.join(Dni, objetosc_wysylek.Dzien_tyg == Dni.id)

In [0]:
display(objetosc_wysylek)

Data,Objetosc_wysylek,Dzien_tyg,id,Day
2022-12-26,635.36,1,1,Poniedziałek
2022-12-19,956.02,1,1,Poniedziałek
2022-12-12,978.25,1,1,Poniedziałek
2022-12-05,945.45,1,1,Poniedziałek
2022-11-28,1102.25,1,1,Poniedziałek
2022-11-21,1236.25,1,1,Poniedziałek
2022-11-14,1025.55,1,1,Poniedziałek
2022-11-07,901.25,1,1,Poniedziałek
2022-10-31,756.25,1,1,Poniedziałek
2022-10-24,802.22,1,1,Poniedziałek


In [0]:
objetosc_wysylek.groupBy('Day')\
    .agg(f.round(f.sum('Objetosc_wysylek'),2).alias('Dzienna_Objetosc_Wysylek'),\
         f.max('Objetosc_wysylek').alias('Max_Objetosc'),
        f.round(f.mean('Objetosc_wysylek'),2).alias('Średnia Objetoc wysyłek'))\
.sort('Średnia Objetoc wysyłek', ascending = True).display()

Day,Dzienna_Objetosc_Wysylek,Max_Objetosc,Średnia Objetoc wysyłek
Niedziela,27913.29,1027.35,134.2
Sobota,115053.45,911.79,553.14
Piątek,118407.3,928.58,566.54
Czwartek,124185.78,879.63,594.19
Środa,129092.47,985.32,617.67
Wtorek,160416.28,1256.63,767.54
Poniedziałek,164840.0,1236.25,792.5


Output can only be rendered in Databricks

In [0]:
### 
objetosc_wysylek.groupBy(f.substring('Data', 1, 4))\
    .agg(f.round(f.sum('Objetosc_wysylek'),2).alias('Dzienna_Objetosc_Wysylek'),\
         f.max('Objetosc_wysylek').alias('Max_Objetosc'),
        f.round(f.mean('Objetosc_wysylek'),2).alias('Średnia Objetoc wysyłek'))\
.sort('Dzienna_Objetosc_Wysylek', ascending = True).withColumnRenamed('substring(Data, 1, 4)', 'Rok').display()

Rok,Dzienna_Objetosc_Wysylek,Max_Objetosc,Średnia Objetoc wysyłek
2019,204669.36,1025.65,560.74
2020,207286.84,1025.56,567.91
2021,209082.69,1256.63,572.83
2022,218869.68,1236.25,599.64


Output can only be rendered in Databricks

In [0]:
dane19_22 = objetosc_wysylek.groupBy(['Data',f.substring('Data', 1, 4)])\
    .agg(f.round(f.sum('Objetosc_wysylek'),2))\
        .sort('Data', ascending=True)

In [0]:
display(dane19_22)

Data,"substring(Data, 1, 4)","round(sum(Objetosc_wysylek), 2)"
2019-01-01,2019,124.56
2019-01-02,2019,639.45
2019-01-03,2019,613.56
2019-01-04,2019,602.23
2019-01-05,2019,589.66
2019-01-06,2019,45.6
2019-01-07,2019,847.45
2019-01-08,2019,856.65
2019-01-09,2019,678.56
2019-01-10,2019,641.25


Output can only be rendered in Databricks